In [53]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import joblib 
from IPython.display import display
from tqdm import tqdm

In [2]:
from utilities import make_model
class feat_test_mdoel(make_model):
    def __init__(self , name , clf , gamma ,x ,y , train_data_loc):
        self.features = x.columns.to_list() 
        self.train_data_loc = train_data_loc
        make_model.__init__(self , name , clf , gamma , x ,y)

In [33]:
labels = ['AGN' , 'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
scores = ['precision' , 'recall' , 'f1_score']
models = ['all_feat_v1' ,'no_color' , 'no_gal_coord' , 'no_IR' , 'no_mw' , 'no_optical_uv']

In [30]:
def score_mean(file):
    all_feat = {
        'AGN' : {
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } ,
        'YSO' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'STAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'HMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'LMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } , 
        'ULX' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'CV':{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'PULSAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        }
    }
    for l in labels:
        for i in range(5):
            model = joblib.load(f'models/{file}_{i}.pkl')
            all_feat[l]['precision'].append(model.result['class_scores']['precision_score'][l])
            all_feat[l]['recall'].append(model.result['class_scores']['recall_score'][l])
            all_feat[l]['f1_score'].append(model.result['class_scores']['f1_score'][l])
    all_mean = {}
    for l in labels:
        all_mean[l] = {}
        for s in scores:
            all_mean[l][s] = np.mean(all_feat[l][s])*100
    all_std = {}
    for l in labels:
        all_std[l] = {}
        for s in scores:
            all_std[l][s] = np.std(all_feat[l][s])*100
    mean_df = pd.DataFrame(all_mean)
    std_df = pd.DataFrame(all_std)
    return mean_df , std_df

In [56]:
final = []
for mod in tqdm(models):
    mean , std = score_mean(mod)
    for l in labels:
        mean[l] = [f'{m:.1f}\pm{s:.2f}' for m,s in zip(mean[l] , std[l])]
    final.append(mean)


100%|██████████| 6/6 [00:15<00:00,  2.56s/it]


In [57]:
final_df = pd.concat(final , keys=models)
final_df

AGN         STAR          YSO         HMXB  \
all_feat_v1   precision  96.8\pm0.14  96.1\pm0.08  92.8\pm0.17  91.8\pm0.24   
              recall     97.6\pm0.15  95.7\pm0.06  95.4\pm0.25  90.7\pm0.66   
              f1_score   97.2\pm0.11  95.9\pm0.07  94.1\pm0.20  91.2\pm0.36   
no_color      precision  96.6\pm0.12  96.0\pm0.17  91.8\pm0.26  91.9\pm0.47   
              recall     97.6\pm0.10  95.2\pm0.14  95.3\pm0.34  90.5\pm0.21   
              f1_score   97.1\pm0.05  95.6\pm0.09  93.5\pm0.17  91.2\pm0.33   
no_gal_coord  precision  93.4\pm0.19  91.6\pm0.22  82.7\pm0.56  67.3\pm0.61   
              recall     93.0\pm0.09  91.6\pm0.22  84.7\pm0.30  69.4\pm0.65   
              f1_score   93.2\pm0.10  91.6\pm0.18  83.7\pm0.42  68.3\pm0.36   
no_IR         precision  95.0\pm0.13  94.2\pm0.13  88.5\pm0.09  91.6\pm0.30   
              recall     96.9\pm0.15  93.2\pm0.07  94.3\pm0.20  90.2\pm0.16   
              f1_score   95.9\pm0.09  93.7\pm0.07  91.3\pm0.14  90.9\pm0.16   
no_mw         precision  91.1\pm0.12  89.2\pm0.18  82.9\pm0.09  92.3\pm0.45   
              recall     95.1\pm0.09  88.2\pm0.11  89.4\pm0.33  90.1\pm0.37   
              f1_score   93.0\pm0.07  88.7\pm0.14  86.0\pm0.15  91.2\pm0.39   
no_optical_uv precision  96.0\pm0.16  95.3\pm0.22  92.4\pm0.18  91.4\pm0.40   
              recall     96.5\pm0.26  94.6\pm0.11  94.9\pm0.15  91.0\pm0.31   
              f1_score   96.3\pm0.17  95.0\pm0.10  93.7\pm0.08  91.2\pm0.27   

                                LMXB          ULX           CV       PULSAR  
all_feat_v1   precision  94.8\pm1.33  71.7\pm1.23  60.6\pm1.16  42.1\pm1.93  
              recall     80.7\pm0.84  71.2\pm1.10  54.5\pm1.50  45.3\pm1.92  
              f1_score   87.2\pm0.60  71.5\pm1.05  57.4\pm1.33  43.7\pm1.89  
no_color      precision  95.0\pm0.61  71.6\pm0.90  61.8\pm1.38  40.4\pm1.35  
              recall     81.5\pm0.56  70.0\pm1.89  55.7\pm1.50  42.4\pm1.92  
              f1_score   87.7\pm0.57  70.7\pm1.28  58.6\pm1.37  41.4\pm1.60  
no_gal_coord  precision  59.3\pm1.50  46.9\pm1.50  43.6\pm1.29  25.5\pm2.19  
              recall     50.8\pm2.24  50.0\pm1.46  33.6\pm1.39  25.7\pm0.89  
              f1_score   54.6\pm1.03  48.4\pm1.44  38.0\pm1.33  25.6\pm1.42  
no_IR         precision  95.6\pm1.30  70.8\pm1.39  58.3\pm0.84  31.3\pm1.28  
              recall     80.8\pm0.34  63.4\pm1.01  50.6\pm1.90  26.9\pm2.02  
              f1_score   87.6\pm0.68  66.9\pm0.91  54.1\pm1.20  28.9\pm1.70  
no_mw         precision  94.5\pm1.02  61.9\pm1.33  56.1\pm2.75  28.2\pm1.86  
              recall     81.7\pm0.52  43.6\pm1.34  45.1\pm1.04  18.6\pm0.74  
              f1_score   87.6\pm0.55  51.2\pm1.36  50.0\pm1.69  22.4\pm0.87  
no_optical_uv precision  94.5\pm1.14  65.5\pm0.82  56.7\pm1.15  35.3\pm1.66  
              recall     81.4\pm0.34  64.9\pm0.30  53.0\pm0.76  40.0\pm1.73  
              f1_score   87.5\pm0.45  65.2\pm0.53  54.8\pm0.67  37.5\pm1.61

In [58]:
print(final_df.to_latex(escape=False , multirow=True))

\begin{tabular}{llllllllll}
\toprule
              &          &          AGN &         STAR &          YSO &         HMXB &         LMXB &          ULX &           CV &       PULSAR \\
\midrule
\multirow{3}{*}{all_feat_v1} & precision &  96.8\pm0.14 &  96.1\pm0.08 &  92.8\pm0.17 &  91.8\pm0.24 &  94.8\pm1.33 &  71.7\pm1.23 &  60.6\pm1.16 &  42.1\pm1.93 \\
              & recall &  97.6\pm0.15 &  95.7\pm0.06 &  95.4\pm0.25 &  90.7\pm0.66 &  80.7\pm0.84 &  71.2\pm1.10 &  54.5\pm1.50 &  45.3\pm1.92 \\
              & f1_score &  97.2\pm0.11 &  95.9\pm0.07 &  94.1\pm0.20 &  91.2\pm0.36 &  87.2\pm0.60 &  71.5\pm1.05 &  57.4\pm1.33 &  43.7\pm1.89 \\
\cline{1-10}
\multirow{3}{*}{no_color} & precision &  96.6\pm0.12 &  96.0\pm0.17 &  91.8\pm0.26 &  91.9\pm0.47 &  95.0\pm0.61 &  71.6\pm0.90 &  61.8\pm1.38 &  40.4\pm1.35 \\
              & recall &  97.6\pm0.10 &  95.2\pm0.14 &  95.3\pm0.34 &  90.5\pm0.21 &  81.5\pm0.56 &  70.0\pm1.89 &  55.7\pm1.50 &  42.4\pm1.92 \\
              & f1_score &  

In [ ]:
mean = mean.reset_index()
display(mean)
print(mean.to_latex(escape=False))